# Esercizio 47 - Shortest path

<img style="width: 800px; margin:auto;" src="https://www.tommasoadamo.it/images/lez19/sp.svg"/>
Shortest path (v0, v2, v4, v3, v5) between vertices v0 and v5 in the weighted directed graph with cost 20.


## Linear programming formulation

Let us consider a directed graph $G = (V, A)$ with the set of nodes $V$ and the set of arcs $A$. 

Let $n$ and $m$ be the cardinality of $V$ and $A$, respectively. A path is a sequence of nodes $v_1, \dots, v_n$, and is said to be elementary if no node appears in the path more than once. 

We denote by $\delta^+(i)$ and $\delta^-(i)$ the set of out-neighbor and in-neighbor of node $i$, respectively.

A standard linear programming formulation to determine a shortest path from node $s$ to node $t$ is the following:


\begin{align}
\min \quad& \sum\limits_{(i,j) \in A} c_{ij} x_{ij} & &\qquad (1)\\
s.t. \quad& \sum\limits_{j \ \in \ \delta^+(i)} x_{ij} - \sum\limits_{j \ \in \ \delta^-(i)} x_{ji} = \begin{cases} 1 &\mbox{if }i = s \\-1 & \mbox{if } i = t\\0  & \mbox{otherwise}\end{cases} & i \in V, &\qquad (2)\\
 \quad& x_{ij} \geq 0& (i, j) \in A. &\qquad(3)
\end{align}


where $c_{ij} \in \mathbb{R}^+$ are the arc costs, and $x_{ij}$ are continuous arc variables. This LP has the special property that it is integral; more specifically, every basic optimal solution (when one exists) has all variables equal to $0$ or $1$.
Constraints (2) are flow conservation constraints.


**Esempi**:

* file "grafo5.csv" con 6 nodi, s = 0 e t = 5
> distanza = 20<br>
> percorso = 0, 2, 4, 3, 5


In [ ]:
# ALERT: execute this cell to prepare input data!
import requests
def download(link, nomeFile=None):
    if nomeFile == None:
        nomeFile = link.split('/')[-1]
    richiesta = requests.get(link)
    if richiesta.status_code == 200:
        with open(nomeFile, 'w') as file:
            file.write(richiesta.text)
            
download('https://tommasoadamo.it/data/grafo5.csv')

In [ ]:
# ALERT: execute this cell to install DOcplex! 
!pip install docplex cplex

In [ ]:
import csv
import docplex.mp.model as cplex

def leggi_grafo(nome):
    with open(nome, 'r') as file:
        reader = csv.reader(file, quoting=2)
        return list(reader)

def shortest_path(c, s, t):
    V = list(range(len(c)))

    with cplex.Model('shortest_path') as mdl:
        x = mdl.continuous_var_matrix(V, V, name="x")

        mdl.minimize(sum(c[i][j] * x[i,j] for i in V for j in V if c[i][j] != 0))

        for i in V:
            if i == s:
                valore = 1
            elif i == t:
                valore = -1
            else:
                valore = 0

            mdl.add_constraint(sum(x[i,j] for j in V if c[i][j] != 0) - sum(x[j,i] for j in V if c[j][i] != 0) == valore)

        mdl.print_information()

        sol = mdl.solve()
        if sol == None:
            print('No solution')
        else:
            print(sol)

            i = s
            path = [s]
            while i != t:
                for j in V:
                    if x[i, j].solution_value > 0:
                        path.append(j)
                        i = j
                        break
            #print(path)
            return path
    return None
    
c = leggi_grafo('grafo5.csv')
p = shortest_path(c, 0, 5)
print('Il percorso più breve è: ')
print(p)       